In [67]:
import numpy as np
import pandas as pd

import scipy
from statsmodels.stats.weightstats import *
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from statsmodels.stats.weightstats import CompareMeans, DescrStatsW


Уровень кальция в крови здоровых молодых женщин равен в среднем 9.5 милиграммам на децилитр и имеет характерное стандартное
отклонение 0.4 мг/дл. В сельской больнице Гватемалы для 160 здоровых беременных женщин при первом обращении для ведения
беременности был измерен уровень кальция; среднее значение составило 9.57 мг/дл. Можно ли утверждать, что средний уровень
кальция в этой популяции отличается от 9.5?

Посчитайте достигаемый уровень значимости. Поскольку известны только среднее и дисперсия, а не сама выборка, нельзя использовать
стандартные функции критериев — нужно реализовать формулу достигаемого уровня значимости самостоятельно.

Округлите ответ до четырёх знаков после десятичной точки.

In [68]:
expected_mean = 9.5
mean = 9.57
st_dev = 0.4
n = 160
z_stat = (mean - expected_mean)/ (st_dev/np.sqrt(n))
z_stat

2.213594362117875

In [69]:
def double_sided_p_value(z_stat):
    return 2 * (1 - scipy.stats.norm.cdf(z_stat))
print('p-value equals to %.4f' % double_sided_p_value(z_stat))

p-value equals to 0.0269


Имеются данные о стоимости и размерах 53940 бриллиантов:

Отделите 25% случайных наблюдений в тестовую выборку с помощью функции sklearn.cross_validation.train_test_split (зафиксируйте random state = 1). На обучающей выборке настройте две регрессионные модели:

1) линейную регрессию с помощью LinearRegression без параметров
2) случайный лес с помощью RandomForestRegressor с random_state=1.

Какая из моделей лучше предсказывает цену бриллиантов? Сделайте предсказания на тестовой выборке, посчитайте модули отклонений предсказаний от истинных цен. Проверьте гипотезу об одинаковом среднем качестве предсказаний, вычислите достигаемый уровень значимости. Отвергается ли гипотеза об одинаковом качестве моделей против двусторонней альтернативы на уровне значимости α=0.05?

In [70]:
data = pd.read_table('diamonds.txt', header=0)
data.head()



C:\Users\1\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


,carat,depth,table,price,x,y,z
0,0.23,61.5,55.0,326,3.95,3.98,2.43
1,0.21,59.8,61.0,326,3.89,3.84,2.31
2,0.23,56.9,65.0,327,4.05,4.07,2.31
3,0.29,62.4,58.0,334,4.20,4.23,2.63
4,0.31,63.3,58.0,335,4.34,4.35,2.75


In [71]:
y = data.price
x = data.drop('price', axis=1)
data.describe()

,carat,depth,table,price,x,y,z
count,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000
mean,0.797940,61.749405,57.457184,3932.799722,5.731157,5.734526,3.538734
std,0.474011,1.432621,2.234491,3989.439738,1.121761,1.142135,0.705699
min,0.200000,43.000000,43.000000,326.000000,0.000000,0.000000,0.000000
25%,0.400000,61.000000,56.000000,950.000000,4.710000,4.720000,2.910000
50%,0.700000,61.800000,57.000000,2401.000000,5.700000,5.710000,3.530000
75%,1.040000,62.500000,59.000000,5324.250000,6.540000,6.540000,4.040000
max,5.010000,79.000000,95.000000,18823.000000,10.740000,58.900000,31.800000


In [72]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

In [73]:
lin_reg = LinearRegression().fit(x_train, y_train)
ran_for_reg = RandomForestRegressor(random_state=1).fit(x_train, y_train)
lin_reg_pred = lin_reg.predict(x_test)
ran_for_reg_pred = ran_for_reg.predict(x_test)
    

In [74]:
abs_error_lr = [abs(i-j) for i, j in zip(y_test, lin_reg_pred)] 
abs_error_rfr = [abs(i-j) for i, j in zip(y_test, ran_for_reg_pred)]
abs_error_lr = np.array(abs_error_lr)
abs_error_rfr = np.array(abs_error_rfr)

In [75]:
stats.ttest_rel(abs_error_lr, abs_error_rfr)

Ttest_relResult(statistic=10.520468419759558, pvalue=8.750308973305836e-26)

In [76]:
print('95%% confidence interval is [%.4f, %.4f]' % DescrStatsW(abs_error_lr - abs_error_rfr).tconfint_mean())

95% confidence interval is [58.9016, 85.8761]
